<a href="https://colab.research.google.com/github/Dongkyu1102/Data_final_project_Naver/blob/main/%EB%B8%8C%EB%9E%9C%EB%93%9C_%ED%86%B5%EA%B3%84%EC%B9%98%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os
import re
import random
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image

In [ ]:
# 폰트 다운로드 런타임 재실행 필수 !!!!!!!!!!! 실행 및 재실행 안하면 시각화 타이틀 부분의 한글 깨짐 !!!!!!!!!!!
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
PATH = '/content/drive/MyDrive/2조_누구냐, 넌!/04. 기타자료/Naver_shopping_크롤링/'

In [ ]:
# 수가공 라벨링 주제분류 데이터
df_subject = pd.read_csv(f"{PATH}real_labeling_topic(1).csv", encoding='utf-8-sig')

# 수가공 라벨링 감정분류 데이터
df_emotion = pd.read_csv(f"{PATH}real_labeling_emotion(1).csv", encoding='utf-8-sig')

In [ ]:
df_subject = df_subject.reset_index(drop=True)
df_emotion = df_emotion.reset_index(drop=True)

# 주제분류 데이터 - df_subject

In [ ]:
 df_subject.rename(columns=
{
    "content": "content",
    "score": "score",
    "date": "date",
    "brand": "brand",
    "가격": "가격(주제)",
    "세정력/거품": "세정력/거품(주제)",
    "향기": "향기(주제)",
    "트러블/진정": "트러블/민감성(주제)",
    "배송/서비스/고객대응": "배송/고객서비스(주제)",
    "탈모 효과 및 쿨링감": "탈모효과(주제)",
    "머릿결*": "머릿결(주제)"
}, inplace=True)

print(df_subject.columns)

Index(['content', 'score', 'date', 'brand', '가격(주제)', '세정력/거품(주제)', '향기(주제)',
       '트러블/민감성(주제)', '배송/고객서비스(주제)', '탈모효과(주제)', '머릿결(주제)'],
      dtype='object')


In [ ]:
df_subject.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5920 entries, 0 to 5919
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   content       5920 non-null   object 
 1   score         5920 non-null   int64  
 2   date          5920 non-null   object 
 3   brand         5920 non-null   object 
 4   가격(주제)        5920 non-null   float64
 5   세정력/거품(주제)    5920 non-null   float64
 6   향기(주제)        5920 non-null   float64
 7   트러블/민감성(주제)   5920 non-null   float64
 8   배송/고객서비스(주제)  5920 non-null   float64
 9   탈모효과(주제)      5920 non-null   float64
 10  머릿결(주제)       5920 non-null   float64
dtypes: float64(7), int64(1), object(3)
memory usage: 508.9+ KB


# 감정분류 데이터 - df_emotion

In [ ]:
 df_emotion.rename(columns=
{
    "content": "content",
    "score": "score",
    "date": "date",
    "brand": "brand",
    "가격": "가격(감정)",
    "세정력/거품": "세정력/거품(감정)",
    "향기": "향기(감정)",
    "트러블/진정": "트러블/민감성(감정)",
    "배송/서비스/고객대응": "배송/고객서비스(감정)",
    "탈모 효과 및 쿨링감": "탈모효과(감정)",
    "머릿결*": "머릿결(감정)"
}, inplace=True)

print(df_subject.columns)

Index(['content', 'score', 'date', 'brand', '가격(주제)', '세정력/거품(주제)', '향기(주제)',
       '트러블/민감성(주제)', '배송/고객서비스(주제)', '탈모효과(주제)', '머릿결(주제)'],
      dtype='object')


In [ ]:
df_emotion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5920 entries, 0 to 5919
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   content       5920 non-null   object 
 1   score         5920 non-null   int64  
 2   date          5920 non-null   object 
 3   brand         5920 non-null   object 
 4   가격(감정)        1209 non-null   float64
 5   세정력/거품(감정)    1754 non-null   float64
 6   향기(감정)        976 non-null    float64
 7   트러블/민감성(감정)   1834 non-null   float64
 8   배송/고객서비스(감정)  711 non-null    float64
 9   탈모효과(감정)      1119 non-null   float64
 10  머릿결(감정)       1069 non-null   float64
dtypes: float64(7), int64(1), object(3)
memory usage: 508.9+ KB


# 기초통계량 확인

In [ ]:
df_subject.describe()

,score,가격(주제),세정력/거품(주제),향기(주제),트러블/민감성(주제),배송/고객서비스(주제),탈모효과(주제),머릿결(주제)
count,5920.000000,5920.000000,5920.000000,5920.000000,5920.000000,5920.000000,5920.000000,5920.000000
mean,3.797804,0.204223,0.296284,0.164865,0.309797,0.120101,0.189020,0.180574
std,1.055238,0.403167,0.456656,0.371090,0.462449,0.325107,0.391558,0.384698
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df_emotion['brand'].value_counts()

라보에이치    1008
반코르       970
닥터방기원     905
보타믹스      572
비어멕       560
이지마인드     480
살림백서      467
트리플에스     400
앙방        360
TS샴푸      198
Name: brand, dtype: int64

In [ ]:
df_emotion.describe()

,score,가격(감정),세정력/거품(감정),향기(감정),트러블/민감성(감정),배송/고객서비스(감정),탈모효과(감정),머릿결(감정)
count,5920.000000,1209.000000,1754.000000,976.000000,1834.000000,711.000000,1119.000000,1069.000000
mean,3.797804,0.864351,0.836374,0.878074,0.723010,0.746835,0.655943,0.782975
std,1.055238,0.342557,0.370041,0.327368,0.447634,0.435131,0.475272,0.412413
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df_emotion.count()

content         5920
score           5920
date            5920
brand           5920
가격(감정)          1209
세정력/거품(감정)      1754
향기(감정)           976
트러블/민감성(감정)     1834
배송/고객서비스(감정)     711
탈모효과(감정)        1119
머릿결(감정)         1069
dtype: int64

In [ ]:
df_emotion['brand'].value_counts()

라보에이치    1008
반코르       970
닥터방기원     905
보타믹스      572
비어멕       560
이지마인드     480
살림백서      467
트리플에스     400
앙방        360
TS샴푸      198
Name: brand, dtype: int64

# 주제분석 데이터 - 빈도수/비율 체크

In [ ]:
print(df_subject['가격(주제)'].value_counts())
print(df_subject['가격(주제)'].value_counts(normalize=True))
print("")
print("")
print(df_subject['세정력/거품(주제)'].value_counts())
print(df_subject['세정력/거품(주제)'].value_counts(normalize=True))
print("")
print("")
print(df_subject['향기(주제)'].value_counts())
print(df_subject['향기(주제)'].value_counts(normalize=True))
print("")
print("")
print(df_subject['트러블/민감성(주제)'].value_counts())
print(df_subject['트러블/민감성(주제)'].value_counts(normalize=True))
print("")
print("")
print(df_subject['배송/고객서비스(주제)'].value_counts())
print(df_subject['배송/고객서비스(주제)'].value_counts(normalize=True))
print("")
print("")
print(df_subject['탈모효과(주제)'].value_counts())
print(df_subject['탈모효과(주제)'].value_counts(normalize=True))
print("")
print("")
print(df_subject['머릿결(주제)'].value_counts())
print(df_subject['머릿결(주제)'].value_counts(normalize=True))

0.0    4711
1.0    1209
Name: 가격(주제), dtype: int64
0.0    0.795777
1.0    0.204223
Name: 가격(주제), dtype: float64


0.0    4166
1.0    1754
Name: 세정력/거품(주제), dtype: int64
0.0    0.703716
1.0    0.296284
Name: 세정력/거품(주제), dtype: float64


0.0    4944
1.0     976
Name: 향기(주제), dtype: int64
0.0    0.835135
1.0    0.164865
Name: 향기(주제), dtype: float64


0.0    4086
1.0    1834
Name: 트러블/민감성(주제), dtype: int64
0.0    0.690203
1.0    0.309797
Name: 트러블/민감성(주제), dtype: float64


0.0    5209
1.0     711
Name: 배송/고객서비스(주제), dtype: int64
0.0    0.879899
1.0    0.120101
Name: 배송/고객서비스(주제), dtype: float64


0.0    4801
1.0    1119
Name: 탈모효과(주제), dtype: int64
0.0    0.81098
1.0    0.18902
Name: 탈모효과(주제), dtype: float64


0.0    4851
1.0    1069
Name: 머릿결(주제), dtype: int64
0.0    0.819426
1.0    0.180574
Name: 머릿결(주제), dtype: float64




# 감정분석 데이터 - 빈도수/비율 체크

In [ ]:
print(df_emotion['가격(감정)'].value_counts())
print(df_emotion['가격(감정)'].value_counts(normalize=True))
print("")
print("")
print(df_emotion['세정력/거품(감정)'].value_counts())
print(df_emotion['세정력/거품(감정)'].value_counts(normalize=True))
print("")
print("")
print(df_emotion['향기(감정)'].value_counts())
print(df_emotion['향기(감정)'].value_counts(normalize=True))
print("")
print("")
print(df_emotion['트러블/민감성(감정)'].value_counts())
print(df_emotion['트러블/민감성(감정)'].value_counts(normalize=True))
print("")
print("")
print(df_emotion['배송/고객서비스(감정)'].value_counts())
print(df_emotion['배송/고객서비스(감정)'].value_counts(normalize=True))
print("")
print("")
print(df_emotion['탈모효과(감정)'].value_counts())
print(df_emotion['탈모효과(감정)'].value_counts(normalize=True))
print("")
print("")
print(df_emotion['머릿결(감정)'].value_counts())
print(df_emotion['머릿결(감정)'].value_counts(normalize=True))

1.0    1045
0.0     164
Name: 가격(감정), dtype: int64
1.0    0.864351
0.0    0.135649
Name: 가격(감정), dtype: float64


1.0    1467
0.0     287
Name: 세정력/거품(감정), dtype: int64
1.0    0.836374
0.0    0.163626
Name: 세정력/거품(감정), dtype: float64


1.0    857
0.0    119
Name: 향기(감정), dtype: int64
1.0    0.878074
0.0    0.121926
Name: 향기(감정), dtype: float64


1.0    1326
0.0     508
Name: 트러블/민감성(감정), dtype: int64
1.0    0.72301
0.0    0.27699
Name: 트러블/민감성(감정), dtype: float64


1.0    531
0.0    180
Name: 배송/고객서비스(감정), dtype: int64
1.0    0.746835
0.0    0.253165
Name: 배송/고객서비스(감정), dtype: float64


1.0    734
0.0    385
Name: 탈모효과(감정), dtype: int64
1.0    0.655943
0.0    0.344057
Name: 탈모효과(감정), dtype: float64


1.0    837
0.0    232
Name: 머릿결(감정), dtype: int64
1.0    0.782975
0.0    0.217025
Name: 머릿결(감정), dtype: float64


# 상관계수

In [ ]:
df_subject.corr()


<ipython-input-80-609c19fc6826>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_subject.corr()


,score,가격(주제),세정력/거품(주제),향기(주제),트러블/민감성(주제),배송/고객서비스(주제),탈모효과(주제),머릿결(주제)
score,1.000000,0.246392,0.083671,0.094634,0.136000,-0.053304,-0.001530,0.174441
가격(주제),0.246392,1.000000,-0.067179,0.005283,-0.168133,-0.020884,-0.059424,-0.089665
세정력/거품(주제),0.083671,-0.067179,1.000000,0.166322,0.026093,-0.146410,-0.025078,0.093548
향기(주제),0.094634,0.005283,0.166322,1.000000,-0.055488,-0.071726,-0.014515,0.081374
트러블/민감성(주제),0.136000,-0.168133,0.026093,-0.055488,1.000000,-0.181219,-0.010882,-0.070439
배송/고객서비스(주제),-0.053304,-0.020884,-0.146410,-0.071726,-0.181219,1.000000,-0.130585,-0.096435
탈모효과(주제),-0.001530,-0.059424,-0.025078,-0.014515,-0.010882,-0.130585,1.000000,0.005538
머릿결(주제),0.174441,-0.089665,0.093548,0.081374,-0.070439,-0.096435,0.005538,1.000000


In [ ]:
df_emotion.corr()

<ipython-input-81-cf319fc70a99>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_emotion.corr()


,score,가격(감정),세정력/거품(감정),향기(감정),트러블/민감성(감정),배송/고객서비스(감정),탈모효과(감정),머릿결(감정)
score,1.000000,0.128391,0.514924,0.348366,0.680668,0.605820,0.729784,0.555792
가격(감정),0.128391,1.000000,0.077152,0.140245,-0.019672,0.102653,0.222562,-0.035828
세정력/거품(감정),0.514924,0.077152,1.000000,0.207008,0.473880,0.316445,0.378538,0.380262
향기(감정),0.348366,0.140245,0.207008,1.000000,0.236887,-0.128037,0.168265,0.328241
트러블/민감성(감정),0.680668,-0.019672,0.473880,0.236887,1.000000,-0.039955,0.482091,0.521066
배송/고객서비스(감정),0.605820,0.102653,0.316445,-0.128037,-0.039955,1.000000,0.303226,0.045835
탈모효과(감정),0.729784,0.222562,0.378538,0.168265,0.482091,0.303226,1.000000,0.473790
머릿결(감정),0.555792,-0.035828,0.380262,0.328241,0.521066,0.045835,0.473790,1.000000
